### Get School Data

In [3]:
#import libraries

import requests
import json
import pandas as pd
from config_school import api_key
import time
import xmltodict
import pprint
from collections import OrderedDict


### Read the CSV file to get the zip codes

In [4]:
zip_file ="../data/zipcodes.csv"
zip_df = pd.read_csv(zip_file,delimiter=",")
zip_df['zip'] = zip_df['zip'].astype(str)
zip_df.head()

,zip,type,decommissioned,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,world_region,country,latitude,longitude,irs_estimated_population_2015
0,29001,STANDARD,0,Alcolu,NaN,NaN,SC,Clarendon County,America/New_York,803,NaN,US,33.76,-80.15,1810
1,29003,STANDARD,0,Bamberg,NaN,Midway,SC,Bamberg County,America/New_York,803,NaN,US,33.29,-81.03,4900
2,29006,STANDARD,0,Batesburg,"Batesburg-leesville, Batsbrg-levil","Holtson Crossroads, Kneece, New Holland Crossr...",SC,Lexington County,America/New_York,803,NaN,US,33.90,-81.54,8070
3,29009,STANDARD,0,Bethune,NaN,NaN,SC,Kershaw County,America/New_York,843,NaN,US,34.41,-80.34,1880
4,29010,STANDARD,0,Bishopville,Wisacky,"Alcot, Ashland, Lucknow, Manville, Mccutchen C...",SC,Lee County,America/New_York,803,NaN,US,34.21,-80.24,8910


### Call the Great School API with zip code
#### The Great School API will return the data in XML format which is converted into dictionary using xmltodict function
#### a) When there are no schools found, dictionary results will be sent as None
#### b) When there is only one school for a zip code, results will be sent as dictionary of dictionary
#### c) When there are more than one schools for a zipcode, results will be sent dictionary of dictionary of list
#### d) if No Great School rating is found for a school, it is defaulted to average rating of 5


In [5]:
base = "https://api.greatschools.org/schools/nearby"
count = 0
school_ratings = {"zip":[],"school name":[],"school_id":[],"rating":[],"type":[],"grade":[]}
default_rating = 5

for zipcode in zip_df["zip"]:
    
    count = count + 1
    
    if count % 20 == 0:
        print(f"{count} nbr of records processed")
    
    params ={"key":api_key, "state":"SC", "zip":zipcode}
    
    time.sleep(5)

    try:
        response=requests.get(base,params=params)
        print(f"{zipcode}:")
        response=response.text
        school_data = xmltodict.parse(response, xml_attribs=True)
#       school_data = json.dumps(school_data, indent=4)
        
#       No Schools found for a zip  
        if school_data['schools'] == None:
            school_ratings["zip"].append(zipcode)
            school_ratings["school name"].append("NaN")
            school_ratings["school_id"].append("NaN")
            school_ratings["type"].append("NaN")
            school_ratings["grade"].append("NaN")
            school_ratings["rating"].append(0)
        else:
#       Only one school found for a zip            
            if (type(school_data['schools']['school']) == type(OrderedDict())):
                try:
                    school_ratings["zip"].append(zipcode)
                    school_ratings["school name"].append(school_data['schools']['school']["name"])
                    school_ratings["school_id"].append(school_data['schools']['school']["gsId"])
                    school_ratings["type"].append(school_data['schools']['school']["type"])
                    school_ratings["grade"].append(school_data['schools']['school']["gradeRange"])
                    school_ratings["rating"].append(school_data['schools']['school']["gsRating"])
                except:
                    school_ratings["rating"].append(default_rating)
            else:
#       More than one school found for a zip            
                for school in school_data['schools']['school']:
                    try:
                        school_ratings["zip"].append(zipcode)
                        school_ratings["school name"].append(school["name"])
                        school_ratings["school_id"].append(school["gsId"])
                        school_ratings["type"].append(school["type"])
                        school_ratings["grade"].append(school["gradeRange"])
                        school_ratings["rating"].append(school["gsRating"])
                    except (KeyError) as err:
                        school_ratings["rating"].append(default_rating)
                            
    except (ConnectionError, TimeoutError, KeyError) as err:
            print(f"Error for zipcode {zip}")
            

29001:
29003:
29006:
29009:
29010:
29014:
29015:
29016:
29018:
29020:
29030:
29031:
29032:
29033:
29036:
29037:
29038:
29039:
29040:
20 nbr of records processed
29042:
29044:
29045:
29047:
29048:
29051:
29052:
29053:
29054:
29055:
29056:
29058:
29059:
29061:
29063:
29065:
29067:
29069:
29070:
29072:
40 nbr of records processed
29073:
29075:
29078:
29080:
29081:
29082:
29101:
29102:
29104:
29105:
29107:
29108:
29111:
29112:
29113:
29114:
29115:
29117:
29118:
29123:
60 nbr of records processed
29125:
29126:
29127:
29128:
29129:
29130:
29133:
29135:
29137:
29138:
29142:
29145:
29146:
29148:
29150:
29152:
29153:
29154:
29160:
29161:
80 nbr of records processed
29162:
29163:
29164:
29166:
29168:
29169:
29170:
29172:
29175:
29178:
29180:
29201:
29203:
29204:
29205:
29206:
29208:
29209:
29210:
29212:
100 nbr of records processed
29214:
29215:
29216:
29217:
29218:
29219:
29220:
29222:
29223:
29225:
29226:
29227:
29229:
29301:
29302:
29303:
29306:
29307:
29316:
29318:
120 nbr of records process

### Converts the school ratings to a DataFrame and ratings to Integer

In [6]:
school_df = pd.DataFrame(school_ratings)
school_df = school_df[["zip","school_id","school name","rating","type","grade"]]
school_df["rating"] = school_df.rating.astype(int)
school_df.head()

,zip,school_id,school name,rating,type,grade
0,29001,1416,Phoenix Charter High School,5,charter,9-12
1,29003,128,Bamberg-Ehrhardt High School,4,public,9-12
2,29003,129,Bamberg-Ehrhardt Middle School,6,public,7-8
3,29003,127,Richard Carroll Elementary,4,public,PK-6
4,29006,730,Batesburg-Leesville Middle School,4,public,6-8


#### Consider only public school for rating and merge with the zips from original file to obtain the Ratings per zip code
#### Zero would be populated on the ratings for zip codes with public school

In [7]:
school_df = school_df[school_df["type"] == "public"]
school_rating_zip = pd.DataFrame(school_df.groupby("zip")["rating"].mean())

school_rating_zip["rating"] = round(school_rating_zip["rating"],1)


school_rating_zip.reset_index(inplace=True)

school_rating_zip = pd.merge(zip_df[["zip"]],school_rating_zip, on="zip",  how="left" )
school_rating_zip = school_rating_zip.fillna(value=0)

school_rating_zip.to_csv("../data/school_rating_zipcode.csv")